Autor: Pablo Manresa Nebot.  
  
Entrenamiento de un modelo de few-shot learning con el algoritmo de MAML de primer orden para el dataset Omniglot.  
  
  **NOTA**: Usualmente este tipo de métodos suele ejecutarse durante unas 10.000 o 20.000 iteraciones, pero debido a las limitaciones técnicas, se ejecutarán unas pocas iteraciones para demostrar como poco a poco va aumentando su capacidad predictiva, pudiendo llegar para este problema en concreto a una exactitud cercana al 99.9% (con las suficientes iteraciones y reproduciendo los parámetros del artículo) para el problema de 5-way, 5-shot tal cual los autores definen en la propia publicación Model-Agnostic Meta-Learning for Fast Adaptation of Deep Networks (https://arxiv.org/abs/1703.03400)

In [1]:
!pip install learn2learn

Se cargan las librerías

In [2]:
import learn2learn as l2l
import numpy as np
import torch
import torchvision

Se definen los parámetros de carga del dataset para transformarlo en un problema de k-shot n-ways learning.

In [3]:
n_ways = 5
k_shots = 5
n_shots_tareas = 2*k_shots
num_tareas = 5000

Se carga el dataset Omniglot

In [4]:
tareas_omniglot = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=n_ways,
                                                  train_samples=n_shots_tareas,
                                                  test_ways=n_ways,
                                                  test_samples=n_shots_tareas,
                                                  num_tasks=num_tareas,
                                                  root='~/datasets/omniglot',
    )

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


El siguiente paso, sería definir una red neuronal que actúe como extractor de características del dataset. Para ello está la opción de definir una red mediante Pytorch, o bien, usar una de las que provee **learn2learn** que está probado que proporcionan buenos resultados en dichos datasets. Por un lado está **OmniglotFC**, que es una red de tipo fully connected, y por otro **OmniglotCNN** que es una red convolucional. Se probará con **OmniglotCNN**. Para ello, el primer paso es definirla:

In [5]:
dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo = l2l.vision.models.OmniglotCNN(output_size=n_ways, hidden_size=128, layers=3)
modelo.to(dispositivo)

OmniglotCNN(
  (base): ConvBase(
    (0): ConvBlock(
      (normalize): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(1, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): ConvBlock(
      (normalize): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (2): ConvBlock(
      (normalize): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
  )
  (features): Sequential(
    (0): Lambda()
    (1): ConvBase(
      (0): ConvBlock(
        (normalize): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (conv): Conv2d(1, 128, kernel_size=(3, 3), stride=(2, 2), pa

Dado que se usará el método **MAML**, se definirá junto a sus parámetros a continuación:

In [6]:
tam_paso_maml = 0.1
MAML = l2l.algorithms.MAML(modelo, tam_paso_maml, first_order=False)
MAML

MAML(
  (module): OmniglotCNN(
    (base): ConvBase(
      (0): ConvBlock(
        (normalize): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (conv): Conv2d(1, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
      (1): ConvBlock(
        (normalize): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
      (2): ConvBlock(
        (normalize): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
    )
    (features): Sequential(
      (0): Lambda()
      (1): ConvBase(
        (0): ConvBlock(
          (normalize): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU()
    

Posteriormente se define el optimizador y la función de pérdida:

In [7]:
tam_paso_adam = 0.01

# Se define como pérdida la entropía cruzada
funcion_perdida = torch.nn.CrossEntropyLoss(reduction='mean')
optimizador = torch.optim.Adam(MAML.parameters(), lr = tam_paso_adam)

Antes de pasar a crear el bucle de entrenamiento, se tendrá que definir una función mediante la cual se produzca la adaptación a un nuevo conjunto de datos. Se definirá siguiendo la estructura sugerida por los mismos creadores de **learn2learn**.

In [8]:
def adaptacion(aprendiz, perdida, imagenes, etiquetas, etapas_adaptacion, k_shots, n_ways, dispositivo):
  imagenes, etiquetas = imagenes.to(dispositivo), etiquetas.to(dispositivo)

  indices_adaptacion = np.zeros(imagenes.size(0), dtype=bool)
  indices_seleccion = np.arange(k_shots * n_ways)*2
  indices_adaptacion[indices_seleccion] = True
  
  indices_evaluacion = torch.from_numpy(~indices_adaptacion)
  indices_adaptacion = torch.from_numpy(indices_adaptacion)

  imagenes_adaptacion = imagenes[indices_adaptacion]
  etiquetas_adaptacion = etiquetas[indices_adaptacion]

  for etapa_adaptacion in range(etapas_adaptacion):
    salida_aprendizaje = aprendiz(imagenes_adaptacion)
    error_train = perdida(salida_aprendizaje, etiquetas_adaptacion)
    aprendiz.adapt(error_train)

  imagenes_evaluacion = imagenes[indices_evaluacion]
  etiquetas_evaluacion = etiquetas[indices_evaluacion]

  predicciones = aprendiz(imagenes_evaluacion)
  error = perdida(predicciones, etiquetas_evaluacion)

  # Se calcula la exactitud
  exactitud = (predicciones.argmax(dim=1).view(etiquetas_evaluacion.shape) == etiquetas_evaluacion).sum().float() / etiquetas_evaluacion.size(0)

  return exactitud, error

El siguiente paso, sería definir el cuerpo de entrenamiento del algoritmo

In [13]:
def entrenamiento(MAML, tareas_omniglot, optimizador, funcion_perdida, dispositivo, config):
  iteraciones_entrenamiento = config["iteraciones_entrenamiento"]
  meta_batch = config["meta_batch"]
  iteraciones_adaptacion = config["iteraciones_adaptacion"]
  k_shots, n_ways = config["k_shots"], config["n_ways"]

  for iteracion in range(iteraciones_entrenamiento):
    # Se limpia la acumulación del gradiente de los pasos anteriores
    optimizador.zero_grad()

    error_meta_train = 0.0
    error_meta_val = 0.0
    exactitud_meta_train = 0.0
    exactitud_meta_val = 0.0

    for tarea in range(meta_batch):
      # Se define un aprendiz para el entrenamiento
      aprendiz = MAML.clone()
      batch_datos = tareas_omniglot.train.sample()
      imagenes, etiquetas = batch_datos

      exactitud, error = adaptacion(aprendiz, funcion_perdida, imagenes, etiquetas, iteraciones_adaptacion, k_shots, n_ways, dispositivo)
      
      error.backward()
      
      error_meta_train += error.item()
      exactitud_meta_train += exactitud.item()

      # Se define un aprendiz para la validación
      aprendiz = MAML.clone()
      batch_datos = tareas_omniglot.validation.sample()
      imagenes, etiquetas = batch_datos

      exactitud, error = adaptacion(aprendiz, funcion_perdida, imagenes, etiquetas, iteraciones_adaptacion, k_shots, n_ways, dispositivo)
      
      error_meta_val += error.item()
      exactitud_meta_val += exactitud.item()

    optimizador.step()
    if config["verbose"] == 1:
      print(f"Iteración {iteracion+1}")
      print(f"Error entrenamiento {error_meta_train/meta_batch}")
      print(f"Exactitud entrenamiento {exactitud_meta_train/meta_batch}")
      print(f"Error validación {error_meta_val/meta_batch}")
      print(f"Exactitud validación {exactitud_meta_val/meta_batch}")
      print("\n\n")
    elif iteracion % 10 == 0:
      print(f"Iteración {iteracion+1}")
      print(f"Error entrenamiento {error_meta_train/meta_batch}")
      print(f"Exactitud entrenamiento {exactitud_meta_train/meta_batch}")
      print(f"Error validación {error_meta_val/meta_batch}")
      print(f"Exactitud validación {exactitud_meta_val/meta_batch}")
      print("\n\n")

    # Se crea el bucle para el test
  error_meta_test, exactitud_meta_test = 0.0, 0.0
  for tarea in range(meta_batch):
    # Se define un aprendiz para el entrenamiento
    aprendiz = MAML.clone()
    batch_datos = tareas_omniglot.test.sample()
    imagenes, etiquetas = batch_datos

    exactitud, error = adaptacion(aprendiz, funcion_perdida, imagenes, etiquetas, iteraciones_adaptacion, k_shots, n_ways, dispositivo)
    
    error_meta_test += error.item()
    exactitud_meta_test += exactitud.item()

  print(f"Error test {error_meta_val/meta_batch}")
  print(f"Exactitud test {exactitud_meta_val/meta_batch}")


config = {}
config["iteraciones_entrenamiento"] = 120
config["meta_batch"] = 32
config["iteraciones_adaptacion"] = 1
config["k_shots"] = k_shots
config["n_ways"] = n_ways
config["verbose"] = 1

entrenamiento(MAML, tareas_omniglot, optimizador, funcion_perdida, dispositivo, config)

Iteración 1
Error entrenamiento 1.2005705144256353
Exactitud entrenamiento 0.487499987706542
Error validación 1.249436704441905
Exactitud validación 0.48124998807907104



Iteración 2
Error entrenamiento 1.241730224341154
Exactitud entrenamiento 0.4812499862164259
Error validación 1.2649349141865969
Exactitud validación 0.4812499904073775



Iteración 3
Error entrenamiento 1.1567829865962267
Exactitud entrenamiento 0.5474999882280827
Error validación 1.217702740803361
Exactitud validación 0.5124999838881195



Iteración 4
Error entrenamiento 1.1596728879958391
Exactitud entrenamiento 0.5299999862909317
Error validación 1.1848582196980715
Exactitud validación 0.4962499886751175



Iteración 5
Error entrenamiento 1.2322745230048895
Exactitud entrenamiento 0.49624998681247234
Error validación 1.2517653685063124
Exactitud validación 0.4637499861419201



Iteración 6
Error entrenamiento 1.1863464582711458
Exactitud entrenamiento 0.49124998645856977
Error validación 1.244258714839816
Exactit

Ahora se probará con un MAML de primer orden, y se comprobará si para el mismo número de iteraciones produce un resultado similar

In [15]:
config = {}
config["iteraciones_entrenamiento"] = 120
config["meta_batch"] = 32
config["iteraciones_adaptacion"] = 1
config["k_shots"] = k_shots
config["n_ways"] = n_ways
config["verbose"] = 1

tam_paso_maml = 0.1
MAML_1st = l2l.algorithms.MAML(modelo, tam_paso_maml, first_order=True)

tam_paso_adam = 0.01

# Se define como pérdida la entropía cruzada
funcion_perdida = torch.nn.CrossEntropyLoss(reduction='mean')

# Se define el optimizador
optimizador_1st = torch.optim.Adam(MAML_1st.parameters(), lr = tam_paso_adam)

entrenamiento(MAML_1st, tareas_omniglot, optimizador_1st, funcion_perdida, dispositivo, config)

Iteración 1
Error entrenamiento 0.8091871570795774
Exactitud entrenamiento 0.6762499809265137
Error validación 0.8850087765604258
Exactitud validación 0.6374999824911356



Iteración 2
Error entrenamiento 0.8862967006862164
Exactitud entrenamiento 0.6574999839067459
Error validación 0.8774734847247601
Exactitud validación 0.6412499845027924



Iteración 3
Error entrenamiento 0.8250762289389968
Exactitud entrenamiento 0.6662499848753214
Error validación 0.8993904097005725
Exactitud validación 0.6562499832361937



Iteración 4
Error entrenamiento 0.812625671736896
Exactitud entrenamiento 0.6837499858811498
Error validación 0.9323175195604563
Exactitud validación 0.6337499814108014



Iteración 5
Error entrenamiento 0.7911419626325369
Exactitud entrenamiento 0.6762499855831265
Error validación 0.8507546801120043
Exactitud validación 0.6549999816343188



Iteración 6
Error entrenamiento 0.8265849584713578
Exactitud entrenamiento 0.6674999776296318
Error validación 0.9569445475935936
Exacti

Con la versión de primer orden se ha conseguido una mejor exactitud, pero sigue estando lejos del 99.9% que los autores de MAML sugieren en la publicación original. No obstante, el número de iteraciones usado es bajo, ya que habitualmente se suelen usar entre 10.000 y 20.000, aquí se han usado 100. Podría probarse otro extractor de características, con el fin de comprobar si mejora la calidad del ajuste.  
  
  Por ejemplo, probando el el extractor `OmniglotFC` proporcionado por learn2learn:

In [42]:
config["iteraciones_entrenamiento"] = 1000
config["verbose"] = 0

modulo_fnn = l2l.vision.models.OmniglotFC(28*28, n_ways)

modulo_fnn.to(dispositivo) 

MAML_1st = l2l.algorithms.MAML(modulo_fnn, tam_paso_maml, first_order=True)
optimizador_1st = torch.optim.Adam(MAML_1st.parameters(), lr = tam_paso_adam)

entrenamiento(MAML_1st, tareas_omniglot, optimizador_1st, funcion_perdida, dispositivo, config)

Iteración 1
Error entrenamiento 1.5654605031013489
Exactitud entrenamiento 0.47624998912215233
Error validación 1.563189648091793
Exactitud validación 0.5012499894946814



Iteración 11
Error entrenamiento 0.7607728745788336
Exactitud entrenamiento 0.7349999789148569
Error validación 0.7851143470034003
Exactitud validación 0.7124999780207872



Iteración 21
Error entrenamiento 0.6471819500438869
Exactitud entrenamiento 0.7612499762326479
Error validación 0.7094968408346176
Exactitud validación 0.7424999829381704



Iteración 31
Error entrenamiento 0.5892453193664551
Exactitud entrenamiento 0.8099999772384763
Error validación 0.5972732221707702
Exactitud validación 0.7874999772757292



Iteración 41
Error entrenamiento 0.5655265841633081
Exactitud entrenamiento 0.8037499748170376
Error validación 0.574047370813787
Exactitud validación 0.7912499792873859



Iteración 51
Error entrenamiento 0.5054878653027117
Exactitud entrenamiento 0.8324999753385782
Error validación 0.5621646400541067
E

Se ha obtenido con 1.000 iteraciones un 89.75% de exactitud con 5-way, 5-shot y una iteración de adaptación.

El bloque `OmniglotFC` está compuesto por:

In [12]:
l2l.vision.models.OmniglotFC(28 ** 2, n_ways)

OmniglotFC(
  (features): Sequential(
    (0): Flatten()
    (1): Sequential(
      (0): LinearBlock(
        (relu): ReLU()
        (normalize): BatchNorm1d(256, eps=0.001, momentum=0.999, affine=True, track_running_stats=False)
        (linear): Linear(in_features=784, out_features=256, bias=True)
      )
      (1): LinearBlock(
        (relu): ReLU()
        (normalize): BatchNorm1d(128, eps=0.001, momentum=0.999, affine=True, track_running_stats=False)
        (linear): Linear(in_features=256, out_features=128, bias=True)
      )
      (2): LinearBlock(
        (relu): ReLU()
        (normalize): BatchNorm1d(64, eps=0.001, momentum=0.999, affine=True, track_running_stats=False)
        (linear): Linear(in_features=128, out_features=64, bias=True)
      )
      (3): LinearBlock(
        (relu): ReLU()
        (normalize): BatchNorm1d(64, eps=0.001, momentum=0.999, affine=True, track_running_stats=False)
        (linear): Linear(in_features=64, out_features=64, bias=True)
      )
   